In [48]:
import os
from dotenv import load_dotenv
import time
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException


In [49]:
def get_messages(driver):
    messages = []
    try:
        # Klicken Sie auf den Instant Messaging Button
        im_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "im2-open-button"))
        )
        print("Instant Messaging Button gefunden")
        im_button.click()
        print("Auf den Instant Messaging Button wurde geklickt")

        # Längere Wartezeit für das Laden des Overlays
        WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.ID, "im2-overlay-wrapper"))
        )
        print("Nachrichtenoverlay wurde geöffnet")

        # Warten auf das Erscheinen der Nachrichtenliste
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "itsl-im2__thread-list"))
        )

        # Scrollen im Overlay
        overlay = driver.find_element(By.ID, "im2-overlay-wrapper")
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", overlay)

        # Warten auf individuelle Nachrichtenthreads
        threads = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "itsl-im2-thread"))
        )

        # Extrahieren der Nachrichteninhalte
        for thread in threads:  # Begrenzen auf die ersten 5 Threads zur Demonstration
            try:
                title = thread.find_element(By.CLASS_NAME, "itsl-im2-thread__title").text
                timestamp = thread.find_element(By.CLASS_NAME, "itsl-im2-thread__timestamp").text
                message = thread.find_element(By.CLASS_NAME, "itsl-im2-thread__text").text
                print(f"\nNachricht von: {title}")
                print(f"Zeitstempel: {timestamp}")
                print(f"Inhalt: {message}...")  # Zeige nur die ersten 100 Zeichen
                messages.append([title, timestamp, message])
            except Exception as e:
                print(f"Fehler beim Extrahieren einer Nachricht: {str(e)}")
        
        return messages

    except TimeoutException:

            print("Konnte den Instant Messaging Button nicht finden oder Drawer nicht öffnen")
            print("Aktuelle Seitenstruktur:")
            print(driver.page_source)

In [50]:
def get_notifications(driver):
    notifications = []
    try:
        # Klicken Sie auf den Benachrichtigungsbutton
        notification_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "Notification"))
        )
        print("Benachrichtigungsbutton gefunden")
        notification_button.click()
        print("Auf den Benachrichtigungsbutton wurde geklickt")

        # Warten auf das Laden des Overlays
        WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.ID, "notifications-overlay-wrapper"))
        )
        print("Benachrichtigungsoverlay wurde geöffnet")

        # Warten auf das Erscheinen der Benachrichtigungsliste
        notifications_list = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "itsl-personal-notifications__list"))
        )
        print("Benachrichtigungsliste gefunden")

        # Scrollen im Overlay
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", notifications_list)
        print("Overlay wurde gescrollt")

        # Kurze Pause, um das Laden der Benachrichtigungen zu ermöglichen
        time.sleep(2)

        # Warten auf individuelle Benachrichtigungen
        notifications_elements = notifications_list.find_elements(By.TAG_NAME, "li")
        print(f"Anzahl gefundener Benachrichtigungen: {len(notifications_elements)}")

        # Extrahieren der Benachrichtigungsinhalte
        for notification in notifications_elements:
            try:
                title_element = notification.find_element(By.CLASS_NAME, "itsl-personal-notification__item__title")
                title = title_element.text
                link = title_element.get_attribute("href")
                info_element = notification.find_element(By.CLASS_NAME, "itsl-personal-notification__item__info")
                info = info_element.text
                
                print(f"\nBenachrichtigung:")
                print(f"Titel: {title}")
                print(f"Link: {link}")
                print(f"Info: {info}")
                
                notifications.append([title, link, info])
            except Exception as e:
                print(f"Fehler beim Extrahieren einer Benachrichtigung: {str(e)}")

        return notifications

    except TimeoutException:
        print("Timeout beim Laden der Benachrichtigungen")
        print("Aktuelle Seitenstruktur:")
        print(driver.page_source)
    except Exception as e:
        print(f"Ein unerwarteter Fehler ist aufgetreten: {str(e)}")
        print("Aktuelle Seitenstruktur:")
        print(driver.page_source)

    return notifications

In [ ]:

# Laden der Umgebungsvariablen aus der .env Datei
load_dotenv()

# Lesen der Umgebungsvariablen
username = os.getenv('ITSLEARNING_USERNAME')
password = os.getenv('ITSLEARNING_PASSWORD')

# Initialisieren Sie den Webdriver (z.B. Chrome)
driver = webdriver.Chrome()



def login(driver, username, password):
    try:
        # Finden Sie das Eingabefeld für den Benutzernamen und geben Sie etwas ein
        username_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "umcLoginUsername"))
        )
        username_field.send_keys(username)

        # Finden Sie das Passwortfeld und geben Sie etwas ein
        password_field = driver.find_element(By.ID, "umcLoginPassword")
        password_field.send_keys(password)

        # Finden Sie den Anmelde-Button und klicken Sie darauf
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'umcLoginFormButton') and .//span[contains(text(), 'Anmelden')]]"))
        )
        login_button.click()

        # Warten Sie auf das Erscheinen des itslearning-Links
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//a[@aria-label='itslearning Neuer Tab']"))
        )
        print("Login erfolgreich")
        return True
    except Exception as e:
        print(f"Login fehlgeschlagen: {str(e)}")
        return False



try:
    # Öffnen Sie die Webseite
    driver.get("https://cloud.schule-mv.de/univention/saml/?location=/univention/portal/")

    # Versuchen Sie sich anzumelden
    if not login(driver, username, password):
        raise Exception("Login fehlgeschlagen")

    # Finden Sie den itslearning-Link
    itslearning_link = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@aria-label='itslearning Neuer Tab']"))
    )
    print("itslearning-Link gefunden")

    # Holen Sie die URL des Links
    itslearning_url = itslearning_link.get_attribute('href')
    print(f"itslearning URL: {itslearning_url}")

    # Öffnen Sie die itslearning-Seite in einem neuen Tab
    driver.execute_script(f"window.open('{itslearning_url}', '_blank');")
    
    # Warten Sie kurz, um sicherzustellen, dass der neue Tab geöffnet wurde
    time.sleep(2)

    # Wechseln Sie zum neuen Tab
    driver.switch_to.window(driver.window_handles[-1])
    
    print(f"Aktuelle URL nach Tab-Wechsel: {driver.current_url}")

    # Warten Sie, bis die Seite vollständig geladen ist
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

    messages = get_messages(driver)
     # Benachrichtigungen abrufen
    notifications = get_notifications(driver)
    print(f"Anzahl abgerufener Benachrichtigungen: {len(notifications)}")

    # Ausgabe der Benachrichtigungen
    for notification in notifications:
        print(f"\nBenachrichtigung:")
        print(f"Titel: {notification[0]}")
        print(f"Link: {notification[1]}")
        print(f"Info: {notification[2]}")

    # Optional: Ausgabe der Benachrichtigungen
    for notification in notifications:
        print(f"\nBenachrichtigung:")
        print(f"Titel: {notification[0]}")
        print(f"Zeitstempel: {notification[1]}")
        print(f"Inhalt: {notification[2][:100]}...")  #
    

except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {str(e)}")

finally:
    # Schließen Sie den Browser
    driver.quit()





In [52]:
def check_for_recent_notifications(notifications: list[list[str]]) -> bool:
    yesterday = datetime.today() - timedelta(days=1)
    wochentag_index = yesterday.date().weekday()
    wochentage = ["Montag", "Dienstag", "Mittwoch", "Donnerstag", "Freitag", "Samstag", "Sonntag"]

    wochentage[wochentag_index]
    recent_notification = False
    for notification in notifications:
        time_text = notification[2]
        if (sum([wochentag in time_text for wochentag in wochentage]) > 0) or ("Vor" in time_text):
            recent_notification = True
    return recent_notification

        

In [53]:
from datetime import datetime, timedelta

In [ ]:
check_for_recent_notifications(notifications)

In [ ]:
messages

In [56]:
def check_for_recent_messages(messages: list[list[str]]) -> bool:
    today_str_part = datetime.today().strftime("%d.%m")
    yesterday_str_part = (datetime.today()-timedelta(days=1)).strftime("%d.%m")
    recent_messages = False
    for message in messages[:5]:
        
        message_date_str = message[1][:5]
        if message_date_str==today_str_part or message_date_str==yesterday_str_part:
            recent_messages = True
    return recent_messages
        


In [57]:
recent_messages = check_for_recent_messages(messages)
recent_notification = check_for_recent_notifications

In [58]:
betreff = "Es gibt weder neue Mitteiligungen noch neue Benachrichtigungen"
if recent_messages or recent_notification:
    betreff = "Es gibt neue Mitteilungen oder Benachrichtungen"

In [65]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import os
from dotenv import load_dotenv

# Laden der Umgebungsvariablen aus der .env Datei
load_dotenv()

# Lesen der Umgebungsvariablen
smtp_server = os.getenv('SMTP_SERVER')
smtp_port = os.getenv('SMTP_PORT')
smtp_username = os.getenv('SMTP_USERNAME')
smtp_password = os.getenv('SMTP_PASSWORD')
email_from = os.getenv('EMAIL_FROM')
email_to = os.getenv('EMAIL_TO').split(', ')

def send_email(betreff, messages, notifications):
    msg = MIMEMultipart()
    msg['From'] = email_from
    msg['To'] = ', '.join(email_to)
    msg['Subject'] = betreff

    body = "Es gibt neue Mitteilungen oder Benachrichtigungen:\n\n"

    if messages:
        body += "Mitteilungen:\n"
        for message in messages:
            body += f"Von: {message[0]}\n"
            body += f"Zeitstempel: {message[1][:5]}\n"
            body += f"Inhalt: {message[2]}\n\n"

    if notifications:
        body += "Benachrichtigungen:\n"
        for notification in notifications:
            body += f"Titel: {notification[0]}\n"
            body += f"Link: {notification[1]}\n"
            body += f"Info: {notification[2]}\n\n"

    msg.attach(MIMEText(body, 'plain'))

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)
    text = msg.as_string()
    server.sendmail(email_from, email_to, text)
    server.quit()

# Aufruf der Funktion
send_email(betreff, messages[:5], notifications[:5])

In [ ]:
messages

In [ ]:
notifications